In [52]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score,mean_absolute_error

In [86]:
df = pd.read_csv('data/kc_house_data.csv')
df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,10/13/2014,221900.0,3,1.00,1180,5650,1.0,NaN,0.0,...,7,1180,0.0,1955,0.0,98178,47.5112,-122.257,1340,5650
1,6414100192,12/9/2014,538000.0,3,2.25,2570,7242,2.0,0.0,0.0,...,7,2170,400.0,1951,1991.0,98125,47.7210,-122.319,1690,7639
2,5631500400,2/25/2015,180000.0,2,1.00,770,10000,1.0,0.0,0.0,...,6,770,0.0,1933,NaN,98028,47.7379,-122.233,2720,8062
3,2487200875,12/9/2014,604000.0,4,3.00,1960,5000,1.0,0.0,0.0,...,7,1050,910.0,1965,0.0,98136,47.5208,-122.393,1360,5000
4,1954400510,2/18/2015,510000.0,3,2.00,1680,8080,1.0,0.0,0.0,...,8,1680,0.0,1987,0.0,98074,47.6168,-122.045,1800,7503


In [87]:
#drop duplicates
df = df.drop_duplicates(subset= ['id'])

In [88]:
#filling nans with zero
df = df.fillna(0.0)

In [89]:
# Replace values in column 'yr_renovated' with booleans based on a condition
df['waterfront'] = df['waterfront'].apply(lambda x: x > 0)
# Replace values in column 'yr_renovated' with booleans based on a condition
df['yr_renovated'] = df['yr_renovated'].apply(lambda x: x > 0)
df = df.rename(columns={'waterfront': 'has_waterfront', 'yr_renovated': 'is_renovated'})

In [90]:
df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,has_waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,is_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,10/13/2014,221900.0,3,1.00,1180,5650,1.0,False,0.0,...,7,1180,0.0,1955,False,98178,47.5112,-122.257,1340,5650
1,6414100192,12/9/2014,538000.0,3,2.25,2570,7242,2.0,False,0.0,...,7,2170,400.0,1951,True,98125,47.7210,-122.319,1690,7639
2,5631500400,2/25/2015,180000.0,2,1.00,770,10000,1.0,False,0.0,...,6,770,0.0,1933,False,98028,47.7379,-122.233,2720,8062
3,2487200875,12/9/2014,604000.0,4,3.00,1960,5000,1.0,False,0.0,...,7,1050,910.0,1965,False,98136,47.5208,-122.393,1360,5000
4,1954400510,2/18/2015,510000.0,3,2.00,1680,8080,1.0,False,0.0,...,8,1680,0.0,1987,False,98074,47.6168,-122.045,1800,7503


In [91]:
df['age'] = 2024 - df['yr_built']
df = df.drop(columns= ['yr_built'])
df.describe()

,id,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,view,condition,grade,sqft_above,zipcode,lat,long,sqft_living15,sqft_lot15,age
count,2.142000e+04,2.142000e+04,21420.000000,21420.000000,21420.000000,2.142000e+04,21420.000000,21420.000000,21420.000000,21420.000000,21420.000000,21420.00000,21420.000000,21420.000000,21420.000000,21420.000000,21420.000000
mean,4.580940e+09,5.407393e+05,3.373950,2.118429,2083.132633,1.512804e+04,1.495985,0.233987,3.410784,7.662792,1791.170215,98077.87437,47.560197,-122.213784,1988.384080,12775.718161,52.907003
std,2.876761e+09,3.679311e+05,0.925405,0.768720,918.808412,4.153080e+04,0.540081,0.765437,0.650035,1.171971,828.692965,53.47748,0.138589,0.140791,685.537057,27345.621867,29.387141
min,1.000102e+06,7.800000e+04,1.000000,0.500000,370.000000,5.200000e+02,1.000000,0.000000,1.000000,3.000000,370.000000,98001.00000,47.155900,-122.519000,399.000000,651.000000,9.000000
25%,2.123537e+09,3.225000e+05,3.000000,1.750000,1430.000000,5.040000e+03,1.000000,0.000000,3.000000,7.000000,1200.000000,98033.00000,47.471200,-122.328000,1490.000000,5100.000000,27.000000
50%,3.904921e+09,4.500000e+05,3.000000,2.250000,1920.000000,7.614000e+03,1.500000,0.000000,3.000000,7.000000,1560.000000,98065.00000,47.572100,-122.230000,1840.000000,7620.000000,49.000000
75%,7.308900e+09,6.450000e+05,4.000000,2.500000,2550.000000,1.069050e+04,2.000000,0.000000,4.000000,8.000000,2220.000000,98117.00000,47.678100,-122.125000,2370.000000,10086.250000,72.000000
max,9.900000e+09,7.700000e+06,33.000000,8.000000,13540.000000,1.651359e+06,3.500000,4.000000,5.000000,13.000000,9410.000000,98199.00000,47.777600,-121.315000,6210.000000,871200.000000,124.000000


In [92]:
df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,has_waterfront,view,...,grade,sqft_above,sqft_basement,is_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,age
0,7129300520,10/13/2014,221900.0,3,1.00,1180,5650,1.0,False,0.0,...,7,1180,0.0,False,98178,47.5112,-122.257,1340,5650,69
1,6414100192,12/9/2014,538000.0,3,2.25,2570,7242,2.0,False,0.0,...,7,2170,400.0,True,98125,47.7210,-122.319,1690,7639,73
2,5631500400,2/25/2015,180000.0,2,1.00,770,10000,1.0,False,0.0,...,6,770,0.0,False,98028,47.7379,-122.233,2720,8062,91
3,2487200875,12/9/2014,604000.0,4,3.00,1960,5000,1.0,False,0.0,...,7,1050,910.0,False,98136,47.5208,-122.393,1360,5000,59
4,1954400510,2/18/2015,510000.0,3,2.00,1680,8080,1.0,False,0.0,...,8,1680,0.0,False,98074,47.6168,-122.045,1800,7503,37


In [93]:

# Calculate IQR for selected columns
Q1 = df.quantile(0.25)
Q3 = df.quantile(0.75)
IQR = Q3 - Q1

# Define outlier boundaries
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Find outliers
outliers = ((df_selected_outliers< lower_bound) | (df_selected_outliers > upper_bound)).any(axis=1)

# Print indices of outliers
outlier_indices = df.index[outliers]

In [94]:
# droping all columns with outliers 
df_selected = df.drop(outlier_indices)


In [96]:
df.corr()

,id,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,has_waterfront,view,condition,grade,sqft_above,is_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,age
id,1.000000,-0.018525,0.000691,0.004343,-0.013451,-0.132648,0.018139,-0.003628,0.011109,-0.024300,0.006470,-0.011794,-0.010864,-0.007934,-0.002766,0.019336,-0.003883,-0.139429,-0.021171
price,-0.018525,1.000000,0.309453,0.526229,0.701875,0.089111,0.256286,0.264898,0.393113,0.034779,0.668020,0.605294,0.118179,-0.052371,0.305744,0.020983,0.584549,0.082438,-0.052906
bedrooms,0.000691,0.309453,1.000000,0.515383,0.579069,0.032490,0.177734,-0.002205,0.078353,0.025893,0.357988,0.480242,0.017204,-0.155061,-0.011266,0.132439,0.394949,0.030570,-0.155241
bathrooms,0.004343,0.526229,0.515383,1.000000,0.755522,0.087779,0.501803,0.063635,0.184949,-0.129362,0.665587,0.686328,0.047259,-0.203884,0.023143,0.223808,0.569453,0.087760,-0.506252
sqft_living,-0.013451,0.701875,0.579069,0.755522,1.000000,0.172586,0.352868,0.104737,0.280839,-0.061677,0.762477,0.876533,0.050623,-0.198995,0.050739,0.240212,0.756186,0.183837,-0.316646
sqft_lot,-0.132648,0.089111,0.032490,0.087779,0.172586,1.000000,-0.005561,0.021429,0.075360,-0.009169,0.113656,0.183210,0.005105,-0.129422,-0.085822,0.230265,0.143805,0.717743,-0.052312
floors,0.018139,0.256286,0.177734,0.501803,0.352868,-0.005561,1.000000,0.020578,0.027175,-0.266859,0.458091,0.522751,0.003063,-0.058532,0.048900,0.124812,0.279071,-0.011367,-0.488935
has_waterfront,-0.003628,0.264898,-0.002205,0.063635,0.104737,0.021429,0.020578,1.000000,0.381654,0.016608,0.082905,0.071737,0.074429,0.029183,-0.012267,-0.037883,0.083921,0.030653,0.024842
view,0.011109,0.393113,0.078353,0.184949,0.280839,0.075360,0.027175,0.381654,1.000000,0.044678,0.247632,0.165289,0.090122,0.087334,0.005835,-0.079019,0.277555,0.073381,0.056171
condition,-0.024300,0.034779,0.025893,-0.129362,-0.061677,-0.009169,-0.266859,0.016608,0.044678,1.000000,-0.150508,-0.161483,-0.055836,0.004372,-0.015566,-0.107154,-0.095256,-0.003450,0.365129


In [101]:
#DAta Preparations
# X3 = df[['bedrooms','bathrooms','sqft_living','grade','condition','sqft_lot','floors','waterfront','yr_built']]
x_model2 = df[['bedrooms','bathrooms','sqft_living','grade','condition','sqft_lot','floors','has_waterfront','age', 'is_renovated']]
x_model2.head()

y_model2= df['price']
y_model2.head()


#Test train split
x_train,x_test,y_train,y_test = train_test_split(x_model2,y_model2,test_size=0.2,random_state=42)


#Modeling
model2 = LinearRegression()
model2.fit(x_train,y_train)

y_pred = model2.predict(x_test)

df_model2= pd.DataFrame({"true":y_test,"pred":y_pred})
df_model2.head()

mse = mean_squared_error(y_test,y_pred)
mae = mean_absolute_error(y_test,y_pred)
r2  = r2_score(y_test,y_pred)

print("mse",mse)
print("mae",mae)
print("R2" ,r2)

mse 42409576968.90263
mae 139947.845465774
R2 0.6497866382685953
